In [1]:
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None
    
    def forward(self,x,y):
        self.x=x
        self.y=y
        out = x*y
        return out
        
    def backward(self,dout):
        dx=dout*self.y
        dy=dout*self.x
        
        return dx,dy
    


In [2]:
apple = 100
apple_num=2
tax=1.1

mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()

# forwardd
apple_price = mul_apple_layer.forward(apple,apple_num)
price =mul_tax_layer.forward(apple_price,tax)
print(price)


220.00000000000003


In [3]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out = x+y
        return out
    def backward(self,dout):
        dx = dout*1
        dy = dout*1
        return dx,dy
    

In [4]:
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1

mul_apple_layer=MulLayer()
mul_orange_layer=MulLayer()
add_apple_orange_layer=AddLayer()
mul_tax_layer=MulLayer()

apple_price = mul_apple_layer.forward(apple,apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
all_price=add_apple_orange_layer.forward(apple_price,orange_price)
price=mul_tax_layer.forward(all_price,tax)

dprice=1
dall_price,dtax=mul_tax_layer.backward(dprice)
dapple_price,dorange_price=add_apple_orange_layer.backward(dall_price)
dorange,dorange_num=mul_orange_layer.backward(dorange_price)
dapple,dapple_num=mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num,dapple,dorange,dorange_num,dtax)


715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [5]:
class Relu:
    def __init__(self):
        self.mask=None
        
    def forward(self,x):
        self.mask=(x<=0)
        out=x.copy()
        out[self.mask]=0
        return out
    
    def backward(self,dout):
        dout[self.mask]=0
        dx=dout
        return dx
    
    

In [7]:
import numpy as np

In [9]:
x = np.array([[1.0, -0.5],[-2.0, 3.0]])
print(x)
mask = (x<=0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [10]:
class Sigmoid:
    def __init__(self):
        self.out=None
    
    def forward(self,x):
        out = 1/(1+np,exp(-x))
        self.out=out
        return out
    def backward(self,dout):
        dx = dout * (1.0-self.out)*self.out
        
        return dx

In [11]:
X = np.random.rand(2,)
W = np.random.rand(2,3)
B = np.random.rand(3)
print(X.shape,W.shape,B.shape)

(2,) (2, 3) (3,)


In [12]:
Y=np.dot(X,W)+B
Y

array([0.51210089, 1.21074042, 0.93430705])

In [22]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [23]:
X_dot_W+B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [24]:
dY = np.array([[1,2,3],[4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [26]:
dB = np.sum(dY,axis=0)
dB

array([5, 7, 9])

In [28]:
class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None
    
    def forward(self,x):
        self.x=x
        out=np.dot(x,self.W)+self.b

        return out
    
    def backward(self,dout):
        dx=np.dot(dout,self.W.T)
        self.dW=np.dot(self.x.T,dout)
        self.db=np.dot(dout,axis=0)
        return dx

### softMax-with-Loss layer

#### digit ->affine->relu->affine->relu->affine->softmax


In [30]:
from common.functions import softmax,cross_entropy_error

In [31]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y=None
        self.t=None
        
    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entorpy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size=self.t.shape[0]
        dx = (self.y-self.t)/batch_size
        return dx


In [33]:
import sys,os
sys.path.append(os.pardir)
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict



In [37]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        self.params={}
        self.params['W1']=weight_init_std*np.random.rand(input_size,hidden_size)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std*np.random.rand(hidden_size,output_size)
        self.params['b2']=np.zeros(output_size)
    
        self.layers=OrderedDict()
        self.layers['Affine1']=Affine(self.params['W1'],self.params['b1'])
        self.layers['Relu1']=Relu()
        self.layers['Affine2']=Affine(self.params['W2'],self.params['b2'])
        self.lastLayer=SoftmaxWithLoss()
    
    def predict(self,x):
        for layer in self.layers.values():
            x=layer.forward(x)
        return x

    def loss(self,x,t):
        y=self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        if t.ndim!=1:t=np.argmax(t,axis=1)
        accuracy=np.sum(y==t)/float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        self.loss(x, t)
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        

In [39]:
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.4826372874778796e-10
b1:2.805146656187883e-09
W2:5.8531130123450875e-09
b2:1.4011236677130556e-07


In [52]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
iters_num=10000
train_size=x_train.shape[0]
batch_size=100
learning_rate=0.1
train_loss_list=[]
train_acc_list=[]
test_acc_list=[]

iter_per_epoch = max(train_size/batch_size,1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    grad = network.gradient(x_batch,t_batch)
    
    for key in ('W1','b1','W2','b2'):
        network.params[key]-=learning_rate*grad[key]
        
    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
    
    

0.11306666666666666 0.1121
0.9051666666666667 0.9072
0.9263333333333333 0.9259
0.9403166666666667 0.9385
0.9449833333333333 0.9428
0.9517 0.9486
0.9575833333333333 0.9533
0.96125 0.9574
0.9642 0.9602
0.9653 0.96
0.9697166666666667 0.9637
0.9686666666666667 0.9639
0.9731833333333333 0.9662
0.9730333333333333 0.9663
0.9766 0.9674
0.9785333333333334 0.9683
0.9785333333333334 0.969
